# 目的
causalでのFine-Tuning、epoch1

ref: 
- https://github.com/unslothai/unsloth
- https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing#scrollTo=6bZsfBuZDeCL
- https://huggingface.co/blog/mlabonne/sft-llama3
- https://huggingface.co/docs/trl/sft_trainer#accelerate-fine-tuning-2x-using-unsloth

In [1]:
# pip install "unsloth[cu121-torch230] @ git+https://github.com/unslothai/unsloth.git"
# -> だめ

# pip uninstall torch
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
# -> OK

In [2]:
# path setting
EXP_NAME = "e012-causal"
# MODEL_NAME = "unsloth/gemma-2-9b-it-bnb-4bit"
MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"

DATA_PATH = "data"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1].replace('.', '-')}"
OUTPUT_PATH = f"output/causal/{EXP_NAME}"
MODEL_OUTPUT_PATH = f"{OUTPUT_PATH}/trained_model"
ENV_PATH = "env_file"

# experiment parameter
DEBUG = False
TRAINING = False
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
REMOVE_LOCAL_FILE = False
WANDB = True

# USE_FOLD = 0
# USE_DATA_RATE = 1.0
# VALID_DATA_SIZE = 3000

# model parameter
TRAINING_MAX_LENGTH = 512
SEED = 42
EPOCH = 1
LR = 2e-04
TRAIN_BS = 1  # 2  # 4  # 16
GRAD_ACC_STEP = 128 // TRAIN_BS  # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 1  # 2  # 4  # 16
# NUM_LABELS = 3

FREEZE_LAYERS = (
    0  # there're 42 layers in total, we don't add adapters to the first 16 layers
)

# rola parameter
LORA_R = 16  # TODO: 64
LORA_ALPHA = LORA_R * 2
LORA_BIAS = "none"

RESUME_FROM_CHECKPOINT = False  # 途中から再開する場合はTrueにする

In [3]:
!nvidia-smi

In [4]:
!python --version

In [5]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/causal":
        print("VastAi! Causal")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}":
        print("VastAi!")
        return base_path
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
OUTPUT_PATH = resolve_path(OUTPUT_PATH)
print(OUTPUT_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)

In [6]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name or "." in dataset_name:
        raise Exception("datasetの名称に'_'、'.'の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# import

In [7]:
import os
import random

import polars as pl
import numpy as np

import wandb

import torch
from datasets import Dataset
from transformers import TrainingArguments

from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

In [8]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = 16  # os.cpu_count()

In [9]:
import torch
import transformers
import datasets
import trl

assert torch.__version__ == "2.4.0+cu121"
assert transformers.__version__ == "4.44.2"
assert datasets.__version__ == "3.0.0"
assert trl.__version__ == "0.11.1"

In [10]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [11]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

# Wandb

In [12]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

'wandb'

# Model Tokenizer Load

In [13]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=TRAINING_MAX_LENGTH,
    dtype=None,
    # load_in_4bit=True,
)

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "embed_tokens",
        "lm_head",
    ],
    lora_alpha=LORA_ALPHA,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=SEED,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

In [15]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(128256, 4096)
          (modules_to_save): ModuleDict(
            (default): Embedding(128256, 4096)
          )
        )
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
        

# Data Import & Preprocess

In [16]:
train_long = pl.read_csv(f"{DATA_PATH}/train_long.csv").filter(
    (pl.col("CorrectAnswer") != pl.col("AnswerAlphabet"))
    & (pl.col("MisconceptionId").is_not_null())
)

In [17]:
if DEBUG:
    train_long = train_long.head(100)

In [18]:
train_long

QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,AnswerType,AnswerText,AllText,AnswerAlphabet,QuestionId_Answer,MisconceptionId,MisconceptionName
i64,str,str,str,str,str,str,str,str,str,str,i64,str
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""\( 3 \times(2+4)-5 \)""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,"""Confuses the order of operatio…"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerAText""","""\( t \)""","""Simplify an algebraic fraction…","""A""","""1000_A""",891,"""Incorrectly cancels what they …"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerCText""","""\( 1 \)""","""Simplify an algebraic fraction…","""C""","""1000_C""",891,"""Incorrectly cancels what they …"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerDText""","""Does not simplify""","""Simplify an algebraic fraction…","""D""","""1000_D""",353,"""Does not recognise when one pa…"
1001,"""Round numbers to two decimal p…","""Rounding to Decimal Places""","""What is \( \mathbf{3 . 5 1 6 3…","""B""","""\( 3.52 \)""","""AnswerAText""","""\( 3.51 \)""","""Round numbers to two decimal p…","""A""","""1001_A""",1379,"""Rounds down instead of up"""
…,…,…,…,…,…,…,…,…,…,…,…,…
99,"""Given the perimeter, work out …","""Volume and Capacity Units""","""A regular pentagon has a total…","""C""","""\( 3.4 \mathrm{~cm} \)""","""AnswerBText""","""\( 12 \mathrm{~cm} \)""","""Given the perimeter, work out …","""B""","""99_B""",1815,"""When given the perimeter of a …"
99,"""Given the perimeter, work out …","""Volume and Capacity Units""","""A regular pentagon has a total…","""C""","""\( 3.4 \mathrm{~cm} \)""","""AnswerDText""","""Not enough information""","""Given the perimeter, work out …","""D""","""99_D""",255,"""Does not know the meaning of t…"
9,"""Identify horizontal translatio…","""Transformations of functions i…","""What transformation maps the g…","""C""","""Translation by vector \( \left…","""AnswerAText""","""Translation by vector \( \left…","""Identify horizontal translatio…","""A""","""9_A""",1889,"""Believes f(x - a) translates t…"


In [19]:
# TODO: CoT(think step by step)をどうFine-Tuningに反映させるべきかを調べる
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Identify the misconception behind Incorrect Answer.

### Input:
Question: {question_text}
Incorrect Answer: {incorrect_answer_text}
Correct Answer: {correct_answer_text}
Construct Name: {construct_name}
Subject Name: {subject_name}

### Response:
{misconception_name}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    texts = []
    for (
        question_text,
        incorrect_answer_text,
        correct_answer_text,
        construct_name,
        subject_name,
        misconception_name,
    ) in zip(
        examples["QuestionText"],
        examples["AnswerText"],  # incorrect_answer_text
        examples["CorrectAnswerText"],
        examples["ConstructName"],
        examples["SubjectName"],
        examples["MisconceptionName"],
    ):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = (
            alpaca_prompt.format(
                question_text=question_text,
                incorrect_answer_text=incorrect_answer_text,
                correct_answer_text=correct_answer_text,
                construct_name=construct_name,
                subject_name=subject_name,
                misconception_name=misconception_name,
            )
            + EOS_TOKEN
        )
        texts.append(text)
    return {
        "text": texts,
    }


train = Dataset.from_polars(train_long).map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/4370 [00:00<?, ? examples/s]

In [20]:
print(train[0]["text"])

In [21]:
print(train[10]["text"])

# Model

In [22]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train,
    dataset_text_field="text",
    max_seq_length=TRAINING_MAX_LENGTH,
    dataset_num_proc=NUM_PROC,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=TRAIN_BS,
        gradient_accumulation_steps=TRAIN_BS,
        warmup_steps=5,
        num_train_epochs=EPOCH,
        learning_rate=LR,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=SEED,
        output_dir=MODEL_OUTPUT_PATH,
        run_name=EXP_NAME,
        save_total_limit=1,
    ),
)

Map (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

In [23]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [24]:
if TRAINING:
    trainer_stats = trainer.train()
else:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_OUTPUT_PATH,
        max_seq_length=TRAINING_MAX_LENGTH,
        dtype=None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
        load_in_4bit=True,  # Use 4bit quantization to reduce memory usage. Can be False.
    )

Step,Training Loss
1,2.785800
2,3.117800
3,2.990700
4,2.606900
5,1.985200
6,1.764800
7,2.019500
8,1.257700
9,1.862900
10,1.756600


# Predict Test

# Save

In [25]:
model.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")

('output/causal/e012-causal/trained_model/lora_model/tokenizer_config.json',
 'output/causal/e012-causal/trained_model/lora_model/special_tokens_map.json',
 'output/causal/e012-causal/trained_model/lora_model/tokenizer.json')

# AWSへのアップロード

In [26]:
# S3へのアップロード
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

In [27]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [28]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system(f"cp {ENV_PATH}/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

0

In [29]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

# ローカルからのデータの削除

In [30]:
# if not DEBUG and REMOVE_LOCAL_FILE:
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [31]:
if WANDB:
    wandb.finish()

In [32]:
print("finish Notebook!")